In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf

In [ ]:
!nvidia-smi

In [ ]:
df = pd.read_csv("/kaggle/input/commonlitreadabilityprize/train.csv")

In [ ]:
df.head()

In [ ]:
df = df.drop(["standard_error"],axis=1)

In [ ]:
df

In [ ]:
import random

In [ ]:
import tensorflow as tf

In [ ]:

ind = random.randint(1,2834-5)

df.iloc[ind].target,df.iloc[ind].excerpt
excerpt = df.iloc[ind].excerpt

In [ ]:
list(excerpt),excerpt.split()

In [ ]:
df["excerpt"] = df["excerpt"].apply(lambda text: text.replace("\n"," "))
df["length"] = df["excerpt"].apply(lambda text: len(text.split()))

In [ ]:
df["length"] = df["excerpt"].apply(lambda excerpt: len(excerpt.split()))
df["char_lengths"] = df["excerpt"].apply(lambda excerpt: len(list(excerpt)))
df["chars"] = df["excerpt"].apply(lambda excerpt: " ".join(list(excerpt)))

In [ ]:
char_lengths = df["char_lengths"].to_numpy()

In [ ]:
lengths = df["length"].to_numpy()

In [ ]:
print(np.percentile(lengths,95))
print(np.percentile(char_lengths,95))

95% of the excerpts are 198 words or less and comprised of 1161 characters or less


In [ ]:
df.describe()

**Create Text Vectorization and Embeddings(character level and word level) and Data Cleaning**

In [ ]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [ ]:
import re

In [ ]:
df.describe()

In [ ]:
max_length = 198
max_char_length = 1161

In [ ]:
len(df)-567

In [ ]:
test_df = df[-567:]
train_df = df[:2267]

In [ ]:
train_excerpts  = train_df["excerpt"].to_numpy()
train_chars = train_df["chars"].to_numpy()
train_targets = train_df["target"].to_numpy()

test_excerpts = test_df["excerpt"].to_numpy()
test_chars = test_df["chars"].to_numpy()
test_targets = test_df["target"].to_numpy()


In [ ]:
train_dataset_values = tf.data.Dataset.from_tensor_slices((train_excerpts,train_chars))
train_dataset_labels = tf.data.Dataset.from_tensor_slices(train_targets)
train_dataset = tf.data.Dataset.zip((train_dataset_values,train_dataset_labels))
train_dataset = train_dataset.batch(32).prefetch(tf.data.AUTOTUNE)

test_dataset_values = tf.data.Dataset.from_tensor_slices((test_excerpts,test_chars))
test_dataset_labels = tf.data.Dataset.from_tensor_slices(test_targets)
test_dataset = tf.data.Dataset.zip((test_dataset_values,test_dataset_labels))
test_dataset = test_dataset.batch(32).prefetch(tf.data.AUTOTUNE)

In [ ]:
train_dataset

In [ ]:
vocab_size = 26924

In [ ]:
import string
alphabet = string.ascii_lowercase + string.digits

In [ ]:
len(alphabet)

In [ ]:
num_alphabet_tokens = len(alphabet) + 2

In [ ]:
vectorizer = TextVectorization(max_tokens=vocab_size,output_sequence_length=max_length)
character_vectorizer = TextVectorization(max_tokens=num_alphabet_tokens,output_sequence_length=1161)
vectorizer.adapt(train_excerpts)
character_vectorizer.adapt(train_chars)
len(vectorizer.get_vocabulary())


In [ ]:
len(character_vectorizer.get_vocabulary())

In [ ]:
#token level embeddings
embedding = tf.keras.layers.Embedding(input_dim=26748,output_dim=128,input_length=max_length)
character_embedding = tf.keras.layers.Embedding(input_dim=38,output_dim=128,input_length=max_char_length)

In [ ]:
from tensorflow.keras import layers
from tensorflow import keras

### inputs = layers.Input(shape=(1,),dtype=tf.string)
x = vectorizer(inputs)
x = embedding(x)
x = layers.Dropout(0.2)(x)
x = layers.Conv1D(filters=64,kernel_size=1,padding="valid")(x)
x = layers.MaxPool1D()(x)
x = layers.Conv1D(filters=32,kernel_size=2,padding="valid")(x)
x = layers.MaxPool1D()(x)
x = layers.Conv1D(filters=32,kernel_size=3,padding="valid")(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1)(x)

In [ ]:
token_inputs = layers.Input(shape=(1,),dtype=tf.string,name="Token Input Layer")
char_inputs = layers.Input(shape=(1,),dtype=tf.string,name="Character Input layer")
token_vectorizer = vectorizer(token_inputs)
token_embeddings = embedding(token_vectorizer)
dropout = layers.Dropout(0.5)(token_embeddings)
x = layers.Bidirectional(layers.LSTM(64,return_sequences=True))(dropout)
x = layers.Bidirectional(layers.LSTM(64))(x)
token_outputs = layers.Dense(128,activation="relu")(x)
token_model = keras.Model(token_inputs,token_outputs)

char_vectorizer = character_vectorizer(char_inputs)
char_embeddings = character_embedding(char_vectorizer)
dropout2 = layers.Dropout(0.5)(char_embeddings)
x = layers.Bidirectional(layers.LSTM(64,return_sequences=True))(dropout2)
x = layers.Bidirectional(layers.LSTM(64))(x)
char_outputs = layers.Dense(128,activation="relu")(x)
char_model = keras.Model(char_inputs,char_outputs)

concat = layers.Concatenate()([token_model.output,char_model.output])
x = layers.Dropout(0.5)(concat)
x = layers.Dense(64,activation="relu")(x)
outputs = layers.Dense(1)(x)


In [ ]:
model_6 = tf.keras.Model([token_model.input,char_model.input],outputs)

In [ ]:
model_6.summary()

In [ ]:
from tensorflow.keras.utils import plot_model

In [ ]:
plot_model(model_6)

In [ ]:
model_6.compile(loss=tf.keras.losses.mse,optimizer=tf.keras.optimizers.Adam(),metrics=tf.keras.metrics.RootMeanSquaredError())

In [ ]:
import tensorflow_hub as hub

In [ ]:
import datetime

In [ ]:
def create_tensorboard_callback(dir_name,experiment_name):
  """
  Creates a TensorBoard callback instand to store log files.
  Stores log files with the filepath:
    "dir_name/experiment_name/current_datetime/"
  Args:
    dir_name: target directory to store TensorBoard log files
    experiment_name: name of experiment directory (e.g. efficientnet_model_1)
  """
  log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=log_dir
  )
  print(f"Saving TensorBoard log files to: {log_dir}")
  return tensorboard_callback

In [ ]:
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath="checkpoint/check.ckpt",monitor="val_root_mean_squared_error",save_best_only=True,save_weights_only=True)

In [ ]:
tf.random.set_seed(42)
model_6.fit(train_dataset,epochs=4,validation_data=(test_dataset),callbacks=[create_tensorboard_callback("CommonLit","CNN-BiLSTM_V2"),model_checkpoint])

In [ ]:
ind = random.randint(1,2834-5)
df.iloc[ind:ind+5]

In [ ]:
df.iloc[0].excerpt

In [ ]:
df_test = pd.read_csv("/kaggle/input/commonlitreadabilityprize/test.csv")

In [ ]:
df_test

In [ ]:
df_test["chars"] = df_test["excerpt"].apply(lambda excerpt: " ".join(list(excerpt)))

In [ ]:
excerpts = df_test["excerpt"].to_numpy()
chars = df_test["chars"].to_numpy()
datasets = tf.data.Dataset.from_tensor_slices((excerpts,chars)).batch(32).prefetch(tf.data.AUTOTUNE)

In [ ]:
chars

In [ ]:
datasets

In [ ]:
preds = model_6.predict([excerpts,chars])

In [ ]:
preds

In [ ]:
submissions = pd.read_csv("/kaggle/input/commonlitreadabilityprize/sample_submission.csv")

In [ ]:
submissions

In [ ]:
submissions["target"] = preds

In [ ]:
submissions

In [ ]:
submissions.to_csv("/kaggle/working/submission.csv",index=False)